In [3]:
import pandas as pd
import numpy as np

# Collaberative-filtering 

In [20]:
df = pd.read_csv("../data/raw/steam_negative_reviews-ENGLISH.csv")

In [21]:
df.head()

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,292030,The Witcher 3: Wild Hunt,85155206,english,They certainly dumbed down the series since th...,1611337043,1611337043,False,0,0,...,True,False,False,76561198010955177,379,73,8966.0,0.0,8966.0,1.514417e+09
1,292030,The Witcher 3: Wild Hunt,85128129,english,"terribly bugs, keeps kicking me out to home sc...",1611299056,1611299056,False,0,0,...,True,False,False,76561198332696736,23,1,8565.0,4973.0,8442.0,1.611364e+09
2,292030,The Witcher 3: Wild Hunt,85109753,english,"While there is a lot of content, the game love...",1611265004,1611265004,False,0,0,...,True,False,False,76561198098346837,507,47,2081.0,0.0,2081.0,1.606027e+09
3,292030,The Witcher 3: Wild Hunt,85088505,english,badbadbadbadbadbadbadbadbadbadbadbadbadbadbadb...,1611235321,1611235321,False,1,0,...,True,False,False,76561199013170832,40,15,1079.0,0.0,1079.0,1.597714e+09
4,292030,The Witcher 3: Wild Hunt,85064415,english,"after serious consideration, I think this game...",1611195190,1611195190,False,1,1,...,True,False,False,76561198085431842,206,65,2031.0,0.0,2031.0,1.594446e+09


In [8]:
df.shape

(1050993, 22)

In [22]:
# sparse matrix

user_game_mat = pd.DataFrame(int(0),index=df['author.steamid'].unique() ,columns=df['app_name'].unique())

reviewers, games= df['author.steamid'], df['app_name']
for r, g in zip(reviewers,games):
    try:
        user_game_mat.loc[int(r),g] = int(1)
    except:
        continue
user_game_mat

,The Witcher 3: Wild Hunt,Half-Life,Counter-Strike: Source,Half-Life 2: Episode Two,Portal 2,X Rebirth,Garry's Mod,Sid Meier's Civilization V,Dead by Daylight,Sid Meier's Civilization VI,...,Thimbleweed Park,Toukiden 2,Dead Rising 4,X-Plane 11,Styx: Shards of Darkness,Train Sim World® 2020,DRAGON QUEST HEROES™ II,Steel Division: Normandy 44,DiRT 4,Half-Life: Alyx
76561198010955177,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76561198332696736,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76561198098346837,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76561199013170832,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76561198085431842,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76561198199668210,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
76561198061774352,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
76561197983355007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
76561198099785821,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## Assign ratings

In [23]:
median_df = df.groupby('app_name')["author.playtime_forever"].median()

In [24]:
median_df

app_name
20XX                                      144.5
A Hat in Time                             492.0
A Short Hike                               86.5
A Way Out                                 343.0
ARK: Survival Evolved                    4398.0
                                          ...  
XCOM 2                                   2391.0
Yakuza 0                                  351.5
Youtubers Life                            711.0
theHunter: Call of the Wild™              828.0
三国群英传8 Heroes of the Three Kingdoms 8     215.5
Name: author.playtime_forever, Length: 315, dtype: float64

In [25]:
df = pd.merge(df, median_df,right_on='app_name', left_on='app_name')

In [26]:
median_df = pd.DataFrame(median_df).reset_index()
median_df.columns = ['app_name', 'median_playtime']


In [27]:
df = pd.merge(df, median_df,right_on='app_name', left_on='app_name')

In [29]:
df.drop(columns=['author.playtime_forever_y'], axis=1, inplace=True)

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever_x,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,median_playtime
0,292030,The Witcher 3: Wild Hunt,85155206,english,They certainly dumbed down the series since th...,1611337043,1611337043,False,0,0,...,False,False,76561198010955177,379,73,8966.0,0.0,8966.0,1.514417e+09,1105.0
1,292030,The Witcher 3: Wild Hunt,85128129,english,"terribly bugs, keeps kicking me out to home sc...",1611299056,1611299056,False,0,0,...,False,False,76561198332696736,23,1,8565.0,4973.0,8442.0,1.611364e+09,1105.0
2,292030,The Witcher 3: Wild Hunt,85109753,english,"While there is a lot of content, the game love...",1611265004,1611265004,False,0,0,...,False,False,76561198098346837,507,47,2081.0,0.0,2081.0,1.606027e+09,1105.0
3,292030,The Witcher 3: Wild Hunt,85088505,english,badbadbadbadbadbadbadbadbadbadbadbadbadbadbadb...,1611235321,1611235321,False,1,0,...,False,False,76561199013170832,40,15,1079.0,0.0,1079.0,1.597714e+09,1105.0
4,292030,The Witcher 3: Wild Hunt,85064415,english,"after serious consideration, I think this game...",1611195190,1611195190,False,1,1,...,False,False,76561198085431842,206,65,2031.0,0.0,2031.0,1.594446e+09,1105.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050988,546560,Half-Life: Alyx,65647678,english,As of day of release:\nSystem: Ryzen 2700; SLI...,1584989199,1584989199,False,8,6,...,False,False,76561198099785821,252,16,174.0,0.0,29.0,1.586467e+09,536.5
1050989,546560,Half-Life: Alyx,65647057,english,Game crashes to desktop cannot play,1584988581,1584988581,False,7,4,...,False,False,76561198895222898,36,3,47.0,0.0,34.0,1.585013e+09,536.5
1050990,546560,Half-Life: Alyx,65647040,english,"im gonna be clear, cant say that its a bad gam...",1584988565,1584988565,False,45,14,...,False,False,76561197984147634,327,2,39.0,0.0,39.0,1.584987e+09,536.5
1050991,546560,Half-Life: Alyx,65646598,english,"As a Half-Life game... I dunno, it just doesn'...",1584988151,1585135046,False,12,1,...,False,False,76561197960598489,490,86,1089.0,0.0,22.0,1.596740e+09,536.5


In [33]:
######################## Assume the ratings  ###############
def get_ratings(df_col):
    df_col['Rating'] = 0
    for i in range(len(df_col)):
        if df_col['author.playtime_forever_x'][i]>= df_col['median_playtime'][i]:
            df_col['Rating'][i] = 5
    #else:
     #   df_col['Rating'][i] = 2
        elif df_col['median_playtime'][i]>df_col['author.playtime_forever_x'][i]>= df_col['median_playtime'][i]*0.8:
            df_col['Rating'][i] = 4
        elif df_col['median_playtime'][i]*0.8>df_col['author.playtime_forever_x'][i]>= df_col['median_playtime'][i]*0.5:
            df_col['Rating'][i] = 3
        elif df_col['median_playtime'][i]*0.5>df_col['author.playtime_forever_x'][i]>= df_col['median_playtime'][i]*0.1:
            df_col['Rating'][i] = 2
        elif df_col['median_playtime'][i]*0.1>df_col['author.playtime_forever_x'][i]:
            df_col['Rating'][i] = 1
    return df_col

In [34]:
get_ratings(df)

/home/abdulium/miniconda3/envs/t5/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/abdulium/miniconda3/envs/t5/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/abdulium/miniconda3/envs/t5/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever_x,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played,median_playtime,Rating
0,292030,The Witcher 3: Wild Hunt,85155206,english,They certainly dumbed down the series since th...,1611337043,1611337043,False,0,0,...,False,76561198010955177,379,73,8966.0,0.0,8966.0,1.514417e+09,1105.0,5
1,292030,The Witcher 3: Wild Hunt,85128129,english,"terribly bugs, keeps kicking me out to home sc...",1611299056,1611299056,False,0,0,...,False,76561198332696736,23,1,8565.0,4973.0,8442.0,1.611364e+09,1105.0,5
2,292030,The Witcher 3: Wild Hunt,85109753,english,"While there is a lot of content, the game love...",1611265004,1611265004,False,0,0,...,False,76561198098346837,507,47,2081.0,0.0,2081.0,1.606027e+09,1105.0,5
3,292030,The Witcher 3: Wild Hunt,85088505,english,badbadbadbadbadbadbadbadbadbadbadbadbadbadbadb...,1611235321,1611235321,False,1,0,...,False,76561199013170832,40,15,1079.0,0.0,1079.0,1.597714e+09,1105.0,4
4,292030,The Witcher 3: Wild Hunt,85064415,english,"after serious consideration, I think this game...",1611195190,1611195190,False,1,1,...,False,76561198085431842,206,65,2031.0,0.0,2031.0,1.594446e+09,1105.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050988,546560,Half-Life: Alyx,65647678,english,As of day of release:\nSystem: Ryzen 2700; SLI...,1584989199,1584989199,False,8,6,...,False,76561198099785821,252,16,174.0,0.0,29.0,1.586467e+09,536.5,2
1050989,546560,Half-Life: Alyx,65647057,english,Game crashes to desktop cannot play,1584988581,1584988581,False,7,4,...,False,76561198895222898,36,3,47.0,0.0,34.0,1.585013e+09,536.5,1
1050990,546560,Half-Life: Alyx,65647040,english,"im gonna be clear, cant say that its a bad gam...",1584988565,1584988565,False,45,14,...,False,76561197984147634,327,2,39.0,0.0,39.0,1.584987e+09,536.5,1
1050991,546560,Half-Life: Alyx,65646598,english,"As a Half-Life game... I dunno, it just doesn'...",1584988151,1585135046,False,12,1,...,False,76561197960598489,490,86,1089.0,0.0,22.0,1.596740e+09,536.5,5


In [43]:
# sparse matrix

user_game_mat_ratings = pd.DataFrame(int(0),index=df['author.steamid'].unique() ,columns=df['app_name'].unique())

reviewers, games, ratings = df['author.steamid'], df['app_name'], df['Rating']
for rev, game, rating in zip(reviewers,games, ratings):
    try:
        user_game_mat_ratings.loc[int(rev),game] = rating
    except:
        continue
user_game_mat_ratings

,The Witcher 3: Wild Hunt,Half-Life,Counter-Strike: Source,Half-Life 2: Episode Two,Portal 2,X Rebirth,Garry's Mod,Sid Meier's Civilization V,Dead by Daylight,Sid Meier's Civilization VI,...,Thimbleweed Park,Toukiden 2,Dead Rising 4,X-Plane 11,Styx: Shards of Darkness,Train Sim World® 2020,DRAGON QUEST HEROES™ II,Steel Division: Normandy 44,DiRT 4,Half-Life: Alyx
76561198010955177,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76561198332696736,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76561198098346837,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76561199013170832,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76561198085431842,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76561198199668210,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
76561198061774352,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
76561197983355007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
76561198099785821,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [41]:
sum(user_game_mat_ratings["The Witcher 3: Wild Hunt"])

0